In [1]:
data_var = '2023-12-17'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9435,2023-12-17,Espanha Acb,08:30,Basquet Girona,Joventut,1.90,1.90,171.5,1.87,1.89,-1.5,2.01,1.78,z9J3VbQP,0.526316,0.526316,0.534759,0.529101,0.052632,0.0,0.0,NaN,NaN,285.386,212.142431,0.743353,175.414,73.397373,0.418424,0.0,0.000,165.85,294.92,0.0,0.0,0.0,0.0,0.000000,0.007522,0.085823,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
9436,2023-12-17,Espanha Acb,08:30,Palencia,Zaragoza,2.07,1.76,160.5,1.82,1.94,1.5,1.90,1.88,S0BVQIYt,0.483092,0.568182,0.549451,0.515464,0.051274,0.0,0.0,NaN,NaN,417.378,212.966298,0.510248,258.334,99.244595,0.384172,0.0,0.000,240.24,206.36,0.0,0.0,0.0,0.0,0.114466,0.045134,0.007483,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
9437,2023-12-17,Espanha Acb,13:00,Breogan,Murcia,2.45,1.56,155.5,1.82,1.96,2.5,2.04,1.72,6iAZPxlm,0.408163,0.641026,0.549451,0.510204,0.049189,0.0,0.0,NaN,NaN,252.340,161.360405,0.639456,141.428,47.514366,0.335962,0.0,0.000,121.36,111.72,0.0,0.0,0.0,0.0,0.313878,0.052378,0.120359,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
9438,2023-12-17,Espanha Acb,14:00,Barcelona,Baskonia,1.33,3.38,166.5,1.83,1.97,-9.5,2.09,1.69,SdLJ9Kbr,0.751880,0.295858,0.546448,0.507614,0.047738,0.0,0.0,NaN,NaN,106.126,17.496968,0.164870,120.492,15.178812,0.125974,0.0,0.000,91.02,113.04,0.0,0.0,0.0,0.0,0.615528,0.052103,0.149652,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
9439,2023-12-17,Espanha Acb,14:30,Valencia,MoraBanc Andorra,1.20,4.57,161.5,1.81,1.97,-11.5,2.07,1.70,AHYiE0jR,0.833333,0.218818,0.552486,0.507614,0.052152,0.0,0.6,NaN,NaN,144.470,51.140529,0.353987,302.108,223.252174,0.738981,0.0,338.488,92.43,256.88,0.0,0.0,0.0,0.0,0.825979,0.059861,0.138795,0.0,0.0,inf,0.0,0.0,0.0,-0.8,-0.16,-22.3125,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9633,2023-12-17,Eua Ncaa,20:00,Texas Southern,Howard,2.10,1.77,141.5,1.80,1.95,1.5,1.95,1.77,vBkKgMON,0.476190,0.564972,0.555556,0.512821,0.041162,0.0,0.0,NaN,NaN,445.122,268.989678,0.604306,206.794,120.106559,0.580803,0.0,0.000,0.00,178.16,0.0,0.0,0.0,0.0,0.120592,0.056569,0.068430,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
9634,2023-12-17,Eua Ncaa,22:30,Jackson State,N. Carolina A&T,1.23,4.44,155.5,1.91,1.91,-10.5,2.00,1.71,j1oZKXUj,0.813008,0.225225,0.523560,0.523560,0.038233,0.0,0.0,NaN,NaN,687.158,605.330908,0.880920,636.190,592.875698,0.931916,0.0,0.000,0.00,1240.00,0.0,1.0,0.0,0.0,0.800639,0.000000,0.110545,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
9635,2023-12-17,Países Baixos Copa Dbl,12:00,Den Bosch,BAL Weert,1.03,11.00,147.5,1.87,1.87,-18.5,1.97,1.74,zsc2Qv52,0.970874,0.090909,0.534759,0.534759,0.061783,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000,89.11,464.75,0.0,0.0,0.0,0.0,1.172046,0.000000,0.087674,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
9636,2023-12-17,Polônia Liga Feminina De Basquete,12:00,Arka Gdynia F,Basket 25 Bydgoszcz F,1.02,12.50,148.5,1.84,1.86,-22.5,1.97,1.71,n3gDZ0cJ,0.980392,0.080000,0.543478,0.537634,0.060392,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000,172.04,161.16,0.0,0.0,0.0,0.0,1.200826,0.007644,0.099917,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
5,00:00,Eua Nba,Sacramento Kings,Utah Jazz,236.5,1.81,0.9987,Over
6,00:30,Eua Nba,Los Angeles Clippers,New York Knicks,219.5,1.98,0.9987,Over
7,17:00,Eua Nba,Boston Celtics,Orlando Magic,226.5,1.89,1.0000,Over
8,17:30,Eua Nba,San Antonio Spurs,New Orleans Pelicans,235.5,1.86,1.0000,Over
10,22:00,Eua Nba,Phoenix Suns,Washington Wizards,242.5,1.80,1.0000,Over
11,23:00,Eua Nba,Portland Trail Blazers,Golden State Warriors,231.5,1.84,0.9999,Over
